# IFT712 - Projet de fin de session

## Sujet

Classification de feuilles d'arbres en utilisant la bibliothèque Sklearn et comparaison des classifieurs présents dans celle-ci pour trouver le modèle adapté à l'étude des plantes. Pour ce faire, on utilisera la validation croisée et la recherche d’hyperparamètres pour chacuns des classifieurs.



## Equipe 8 :


___
___

# 0. Configuration

In [1]:
'''
 Imporation of python libraries
'''
from LogisticRegression_classifieur import LogisticRegressionClassifier
from knn_classifier import KNNClassifier
from decision_trees_classifier import DTSClassifier
from adaboost_classifieur import adaboostClassifier
from svm_classifieur import svmClassifier
from neural_network_classifier import NNClassifier
from random_forest_classifieur import RandForestClassifier
from data_manager import data_extract
from data_manager import data_cleaning
from data_manager import generate_combinations_for_attributes
from data_manager import to_array
from data_manager import ACP
from sklearn.metrics import accuracy_score as ac
from sklearn.model_selection import KFold 
import utils_affichage
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

___
___

# 1. Préparation des données

On utilise les données de classification de feuilles du site [Kaggle](https://www.kaggle.com/c/leaf-classification/data).
Les données sont composées de 2 ensembles (train et test). 

Chaque échantillons est caractérisé par :
- un id unique par image 
- 64 attributs pour la marge de la feuille
- 64 attributs pour la forme de la feuille
- 64 attributs pour la texture de la feuille

In [2]:
data_path_train = './data/train.csv'
data_path_test = './data/test.csv'

#Extract data for training dataset
data_attributes_train, data_train_df, ids_train, labels = data_extract(data_path_train,dataset = "train")

#Extract data for testing dataset
res = data_extract(data_path_test,dataset = "test")
data_attributes_test = res[0]
data_test_df = res[1]
ids_test = res[2]

#Clean data
#print(data_train_df)

max_len_generated_subset = 4

#generate all combinations for attributes
generated_subsets = generate_combinations_for_attributes(data_attributes_train,max_len_generated_subset)



In [2]:
# Dataset Preparation
# Training data 
data_train = pd.read_csv("./data/train.csv")
data_train.set_index("id", inplace=True)

# Test data
data_test = pd.read_csv("./data/test.csv")
data_test.set_index("id", inplace=True)

display(data_train)

# Separation of caracteristiques/tragets of data
# Training data 
y_train = data_train["species"]
X_train = data_train.drop(["species"], axis=1)

# Test data (no target column)
X_test = data_test.copy()

# Encoding "One-hot" for training targets 
class_names = np.unique(y_train)
lblBin = LabelBinarizer().fit(y_train)

#Data normalization
# Training data 
mean = np.mean(X_train, axis=0)
std = np.std(X_train, axis=0)
X_train = (X_train - mean) / std

# Test data
X_test = (X_test - mean) / std

# Splitting the training data for validation
val_perc = 0.3
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=val_perc, random_state=0, shuffle=True)

# Encoding targets "one-hot"
y_train_not_bin = y_train.copy()
y_train = lblBin.transform(y_train)
y_val_not_bin = y_val.copy()
y_val = lblBin.transform(y_val)


,species,margin1,margin2,margin3,margin4,margin5,margin6,margin7,margin8,margin9,...,texture55,texture56,texture57,texture58,texture59,texture60,texture61,texture62,texture63,texture64
id,,,,,,,,,,,,,,,,,,,,,
1,Acer_Opalus,0.007812,0.023438,0.023438,0.003906,0.011719,0.009766,0.027344,0.0,0.001953,...,0.007812,0.000000,0.002930,0.002930,0.035156,0.000000,0.000000,0.004883,0.000000,0.025391
2,Pterocarya_Stenoptera,0.005859,0.000000,0.031250,0.015625,0.025391,0.001953,0.019531,0.0,0.000000,...,0.000977,0.000000,0.000000,0.000977,0.023438,0.000000,0.000000,0.000977,0.039062,0.022461
3,Quercus_Hartwissiana,0.005859,0.009766,0.019531,0.007812,0.003906,0.005859,0.068359,0.0,0.000000,...,0.154300,0.000000,0.005859,0.000977,0.007812,0.000000,0.000000,0.000000,0.020508,0.002930
5,Tilia_Tomentosa,0.000000,0.003906,0.023438,0.005859,0.021484,0.019531,0.023438,0.0,0.013672,...,0.000000,0.000977,0.000000,0.000000,0.020508,0.000000,0.000000,0.017578,0.000000,0.047852
6,Quercus_Variabilis,0.005859,0.003906,0.048828,0.009766,0.013672,0.015625,0.005859,0.0,0.000000,...,0.096680,0.000000,0.021484,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.031250
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1575,Magnolia_Salicifolia,0.060547,0.119140,0.007812,0.003906,0.000000,0.148440,0.017578,0.0,0.001953,...,0.242190,0.000000,0.034180,0.000000,0.010742,0.000000,0.000000,0.000000,0.000000,0.018555
1578,Acer_Pictum,0.001953,0.003906,0.021484,0.107420,0.001953,0.000000,0.000000,0.0,0.029297,...,0.170900,0.000000,0.018555,0.000000,0.011719,0.000000,0.000000,0.000977,0.000000,0.021484
1581,Alnus_Maximowiczii,0.001953,0.003906,0.000000,0.021484,0.078125,0.003906,0.007812,0.0,0.003906,...,0.004883,0.000977,0.004883,0.027344,0.016602,0.007812,0.000000,0.027344,0.000000,0.001953


___
___

# 2. Classification

Dans cette partie nous allons mettre en place 6 classifieurs :
- Random Forest
- Neural Network
- SVM
- AdaBoost
- Decision Tree
- KNeighbors
- Regression linéaire


In [5]:
# Resultat for Visual Comparison
accu_dict = {}

# Dictionary  of labels for test preditions and list of score fonctions
pred_dict = {}
scorers = {"Accuracy": 'accuracy', "Precision": 'precision_micro', "F1": 'f1_micro'}


___


## Classifieur par Random Forest
Une `random forest` est un estimateur qui utilise plusieurs arbres de décision pour palier à la tendance de sur-apprentissage de ceux-ci. Chaque arbre apprend sur les données d'entrée avec des paramètres différents puis l'estimateur classe les points par la probabilité la plus haute pour tout les arbres.


___

## Classifieur par réseau de neurones
Un réseau de neurones est un algorithme d’apprentissage supervisé de plusiers fonctions d’apprentissage. Il calcule la prédiction et la compare avec la valeur de y à l'aide de la fonction de coût et l'erreur est propagé en arrière afin de mettre à jour les poids des différentes fonctions.

### NN - Entrainement

On entraine maintenant le modèle sur les données completes.

In [ ]:
model = NNClassifier(X_train, y_train, X_val, y_val, class_names, scorers)
model.train_default(verbose=True)

### NN - Recherche hyper-paramètres

On fait une recherche d'hyper-paramètres sur :
- `hidden_layer_sizes` : Taille des couches cachées du réseau
- `solveur` : Méthode d'optimisation des poids `W`
- `learning_rate_init` : Longueur initial des pas lors de la descente de gradient
- `alpha` : Terme de régularisation L2, correspond à la pénalité sur la taille des poids dans `W`

In [ ]:
# recherche hyperparametre
NN_param = {
    'hidden_layer_sizes': [(50,), (100,), (200,)],
    'solver': ['lbfgs', 'sgd', 'adam'],
    'learning_rate_init': [1e-2, 1e-1, 1.0, 10.0],
    'alpha': [1e-1, 1e-2, 1e-3, 1e-4, 1e-5, 1e-6]
}

model = NNClassifier(X_train, y_train, X_val, y_val, class_names, scorers)
accu_train_hyperparm, accu_val_hyperparm = model.train_hyperparameter(NN_param, random_search=True, verbose=True)

In [ ]:
# Append accuracy percentage for visual comparison
accu_dict["Neural network"] = accu_val_hyperparm*100


### NN - Prédiction des valeurs de test

On peut alors prédire les classes pour l'ensemble de données de test.

In [ ]:
NN_pred = model.predict(X_test)
NN_pred_label = lblBin.inverse_transform(NN_pred)

pred_dict["Neural network"] = NN_pred_label
df_pred = pd.DataFrame.from_dict(pred_dict)
display(df_pred)

### NN - Affichage des courbes d'apprentissage

On peut afficher les courbes d'apprentissage du modèle si on ne fait varier qu'un seul hyper parametre par une recherche complete.

In [ ]:
NN_param = {
    'learning_rate_init': np.arange(0.0001, .5, 0.1),
    'solver': ['sgd'],
}
model = NNClassifier(X_train, y_train, X_val, y_val, class_names, scorers)
model.train_hyperparameter(NN_param, random_search=False, verbose=True)


In [ ]:
aff_NN = utils_affichage.Affichage(model)

# display(aff_NN.hyper_results())

aff_NN.best_estimators(n_top=2)

y_val_pred = model.predict(X_val)
display(aff_NN.class_report(y_val, y_val_pred, mean_only=True))

aff_NN.plot(param_abscissa="learning_rate_init")

___


## Classifieur par SVM
Les machines à vecteurs support (SVM) sont des algorithmes d’apprentissage supervisé  utilisées pour la classification et la régression. Elles reposent souvent sur l’utilisation de noyaux qui permettent de transformer l’espace de représentation des données d’entrées en un espace de plus grande dimension, dans lequel il existe une séparation linéaire, le but est de maximiser la distance entre les groupes de données.

___


## Classifieur par Adaboost
L’adaptative boosting est un algorithme de boosting qui regroupe plusieurs classifieurs faibles en leur donnant un poids selon leurs performances pour en faire un classifieur fort.

### Adaboost - Entrainement

On entraine sans la recherche d hyper-parametres.

In [ ]:
model = adaboostClassifier(X_train, y_train, X_val, y_val, class_names, scorers)
accu_train, accu_val = model.train_sans_grid()

print('Accuracy train: {:.3%}'.format(accu_train))
print('Accuracy validation: {:.3%}'.format(accu_val))


### Adaboost - Recherche hyper-paramètres

On fait une recherche d'hyper-paramètres sur :
- `estimator__n_estimators` :  nombre de modèles à entraîner de manière itérative
- `estimator__learning_rate` : contribution de chaque modèle aux pondérations

In [ ]:
adaboost_param = {
    'estimator__n_estimators': np.arange(10, 100, 10),
    'estimator__learning_rate': [0.01, 0.05, 0.1, 1],
}

model = adaboostClassifier(X_train, y_train, X_val, y_val, class_names, scorers)
accu_train_hyperparm, accu_val_hyperparm, best_estimator, best_accu = model.train(adaboost_param, random_search=True)

In [ ]:
print('best_cross_val_acc : {}'.format(best_accu))
print('\nBest estimator:\n{}'.format(best_estimator))

print("[Estimator parameters]")
display(model.estimator.get_params())

print('Accuracy train: {:.3%}'.format(accu_train_hyperparm))
print('Accuracy validation: {:.3%}'.format(accu_val_hyperparm))

In [ ]:
# Append accuracy percentage for visual comparison
accu_dict["AdaBoost"] = accu_val_hyperparm*100

### Adaboost - Prédiction des valeurs de test

On peut alors prédire les classes pour l'ensemble de données de test.

In [ ]:
adaboost_pred = model.predict(X_test)
adaboost_pred_label = lblBin.inverse_transform(adaboost_pred)

pred_dict["AdaBoost"] = adaboost_pred_label
df_pred = pd.DataFrame.from_dict(pred_dict)
display(df_pred)


### Adaboost - Affichage des courbes d'apprentissage

On peut afficher les courbes d'apprentissage du modèle si on ne fait varier qu'un seul hyper parametre par une recherche complete.

In [ ]:
Ada_param = {
    'estimator__n_estimators': np.arange(10, 100, 10)
}
model = adaboostClassifier(X_train, y_train, X_val, y_val, class_names, scorers)
model.train(Ada_param, random_search=False)


In [ ]:


aff_Ada = utils_affichage.Affichage(model)

#display(aff_Ada.hyper_results())

aff_Ada.best_estimators(n_top=2)

y_val_pred = model.predict(X_val)
display(aff_Ada.class_report(y_val, y_val_pred, mean_only=True))

aff_Ada.plot(param_abscissa="estimator__n_estimators")

___


## Classifieur par Decision trees
Une `decision trees` est un méthode d’apprentissage supervisé non paramétrique pour la classification et la régression. Notre objectif est de créer un modèle qui prédit la valeur de la variable cible en apprenant des règles de décision simples déduites des caractéristiques des données. L’arbre peut être considéré comme une approximation constante par segments.


In [ ]:
decision_tree_classifier = DTSClassifier()
depth_max = 20
min_samples_split = [2,3,4]
criterion = ["gini","entropy"]
min_samples_leaf = [1,2]
accuracy, f1, precision = decision_tree_classifier.cross_validation(data_train_df[:500],labels[:500],range(1,depth_max),min_samples_split,criterion,min_samples_leaf)

print("Accuracy ",accuracy)
print("f1 ",f1)
print("precision ",precision)

predictions = decision_tree_classifier.predict(data_train_df[500:])
print(ac(predictions,labels[500:]))

Using complet search:
Fitting 5 folds for each of 228 candidates, totalling 1140 fits


/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWar

/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWar

/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWar

/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/home/yoann/.local/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1334: UndefinedMetricWar

In [6]:
# Append accuracy percentage for visual comparison
accu_dict["DTS"] = accuracy*100

In [12]:
decisiontree_pred_label = decision_tree_classifier.predict(data_test_df)

pred_dict["DTS"] = decisiontree_pred_label

df_pred = pd.DataFrame.from_dict(pred_dict)
display(df_pred)


,DTS
0,Quercus_Trojana
1,Tilia_Tomentosa
2,Acer_Circinatum
3,Quercus_Coccinea
4,Acer_Rufinerve
...,...
589,Acer_Circinatum
590,Alnus_Rubra
591,Liquidambar_Styraciflua
592,Quercus_Phillyraeoides


In [14]:


aff_DTS = utils_affichage.Affichage(decision_tree_classifier)

#display(aff_Ada.hyper_results())

aff_DTS.best_estimators(n_top=2)

y_val_pred = decision_tree_classifier.predict(X_val)
display(aff_DTS.class_report(y_val, y_val_pred, mean_only=True))

aff_DTS.plot(param_abscissa="max_depth")

AttributeError: 'DTSClassifier' object has no attribute 'hyper_search'

___


## Classifieur par KNeighbors
Une `KNeighbors` est une méthode d’apprentissage basé sur des exemples ou d’apprentissage non généralisé. Elle ne cherche pas à construire un modèle interne général, mais simplement à stocker des exemples de données de formation. La classification est calculée à la majorité simple des voisins les plus proches de chaque point: une classe de données est attribuée à un point de requête qui a le plus de représentants parmi les voisins les plus proches de ce point.


___


## Classifieur par regression logistique 
La régression logistique est un algorithme d’apprentissage automatique utilisé dans les problèmes de classification et qui a pour but attribuer des observations à un ensemble discret de classes, c’est un modèle linéaire. Grâce à la fonction sigmoïde logistique, la régression logistique transforme sa sortie en une probabilité d'appartenir à l'une ou à l'autre classe dans le cas binaire.

___
___

# 3. Comparaison des résultats

On compile les résultats des classifieurs étudiés pour les comparer sur leur 'accuracy'.



In [ ]:
___
___

# 4. Conclusions

